In [1]:
import pandas as pd
import os

# Define the folder paths
folder_2000_2005 = "2000 to 2005 ACCIDENT"  # Replace with the actual path
folder_2006_2020 = "Accident"  # Replace with the actual path

# List of CSV files to merge
csv_files = [
    "accident",
    "person",
    "vehicle",
    "accident_event",
    "ACCIDENT_LOCATION",
    "road_surface_cond",
    "atmospheric_cond",
    "subdca",
    "accident_chainage",
    "Node",
    "NODE_ID_COMPLEX_INT_ID"
]

# Dictionary to hold merged dataframes
merged_data = {}

# Loop through each CSV file and merge
for csv_file in csv_files:
    # Construct the full path for each file in both folders
    path_2000_2005 = os.path.join(folder_2000_2005, csv_file + ".csv")
    path_2006_2020 = os.path.join(folder_2006_2020, csv_file + ".csv")
    
    # Read the CSVs
    df_2000_2005 = pd.read_csv(path_2000_2005)
    df_2006_2020 = pd.read_csv(path_2006_2020)
    
    # Concatenate the DataFrames
    merged_df = pd.concat([df_2000_2005, df_2006_2020], ignore_index=True)
    
    # Store the merged DataFrame in the dictionary
    merged_data[csv_file] = merged_df

C:\Users\buida\AppData\Local\Temp/ipykernel_21356/2748065923.py:33: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2000_2005 = pd.read_csv(path_2000_2005)
C:\Users\buida\AppData\Local\Temp/ipykernel_21356/2748065923.py:34: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2006_2020 = pd.read_csv(path_2006_2020)
C:\Users\buida\AppData\Local\Temp/ipykernel_21356/2748065923.py:33: DtypeWarning: Columns (0,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2000_2005 = pd.read_csv(path_2000_2005)
C:\Users\buida\AppData\Local\Temp/ipykernel_21356/2748065923.py:34: DtypeWarning: Columns (6,9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2006_2020 = pd.read_csv(path_2006_2020)
C:\Users\buida\AppData\Local\Temp/ipykernel_21356/2748065923.py:33: DtypeWarning: Columns (0,3,11,36) have mixed types. Specify dtype

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert ACCIDENTDATE to datetime format
merged_data["accident"]['ACCIDENTDATE'] = pd.to_datetime(merged_data["accident"]['ACCIDENTDATE'], dayfirst=True)

# Remove data for years 2000 and 2020
filtered_data_1 = merged_data["accident"][~merged_data["accident"]['ACCIDENTDATE'].dt.year.isin([2000, 2020])]

# 2. Change the label of Severity
severity_mapping = {
    1: 'Fatal accident',
    2: 'Serious injury accident',
    3: 'Other injury accident',
    4: 'Non injury accident'
}
filtered_data_1['SEVERITY'] = filtered_data_1['SEVERITY'].map(severity_mapping)

# Group by month and severity and count the number of accidents
yearly_accidents = (filtered_data_1.groupby([filtered_data_1['ACCIDENTDATE'].dt.to_period("Y"), 'SEVERITY'])
                     .size()
                     .unstack()
                     .fillna(0))

# Convert Period index back to datetime format for plotting
yearly_accidents.index = yearly_accidents.index.to_timestamp()


# Plot
plt.figure(figsize=(15, 8))
for severity, values in yearly_accidents.items():
    # Check if there are any non-zero values for the severity
    if values.sum() > 0:
        plt.plot(values, label=f'Severity: {severity}')

plt.title('Number of Accidents Over Time by Severity', fontsize=18)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Number of Accidents', fontsize=16)
plt.legend(fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Filtering the data based on the specified years
filtered_data_2 = merged_data["accident"][~merged_data["accident"]['ACCIDENTDATE'].dt.year.isin([2000, 2005, 2006, 2020])]

# Grouping by month and severity to calculate the number of accidents for each month
monthly_crash_frequency = filtered_data_2.groupby([filtered_data_2['ACCIDENTTIME'].dt.hour, 'SEVERITY']).size().unstack().fillna(0)

# Plotting the stacked bar chart
plt.figure(figsize=(15, 8))
monthly_crash_frequency.plot(kind='bar', stacked=True, figsize=(15, 8), colormap='viridis')

plt.title('Monthly Crash Frequency by Severity (Excluding years 2000, 2005, 2006, 2020)', fontsize=18)
plt.xlabel('Month', fontsize=16)
plt.ylabel('Number of Accidents', fontsize=16)
plt.xticks(ticks=range(24), rotation=0)
plt.legend(fontsize=14,loc='upper center', ncol=4)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Filtering the data based on the specified years
filtered_data_3 = merged_data["accident"]

# Function to convert time to hour
def convert_time(time):
    if isinstance(time, str):  # Check if the value is a string
        time = time.strip()  # Remove any extra spaces
        try:
            return pd.to_datetime(time, format='%H:%M:%S').hour
        except ValueError:
            return None
    else:
        return None

filtered_data_3['ACCIDENTTIME'] = filtered_data_3['ACCIDENTTIME'].apply(convert_time)

# Grouping by hour and severity to calculate the number of accidents for each hour
hourly_crash_frequency = filtered_data_3.groupby([filtered_data_3['ACCIDENTTIME'], 'SEVERITY']).size().unstack().fillna(0)

# Plotting the stacked bar chart
plt.figure(figsize=(15, 8))
hourly_crash_frequency.plot(kind='bar', stacked=True, figsize=(15, 8), colormap='viridis')

plt.title('Hourly Crash Frequency by Severity', fontsize=18)
plt.xlabel('Hour', fontsize=16)
plt.ylabel('Number of Accidents', fontsize=16)
plt.xticks(ticks=range(24), labels=[str(i) for i in range(24)], rotation=0)
plt.legend(fontsize=14,loc='upper center', ncol=4)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Grouping by Day of Week and severity to calculate the number of accidents for each day
daily_crash_frequency = filtered_data_2.groupby([filtered_data_2['Day Week Description'], 'SEVERITY']).size().unstack().fillna(0)

# Define an ordered category for the days of the week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_crash_frequency = daily_crash_frequency.reindex(day_order)

# Plotting the stacked bar chart
plt.figure(figsize=(15, 8))
daily_crash_frequency.plot(kind='bar', stacked=True, figsize=(15, 8), colormap='viridis')

plt.title('Daily Crash Frequency by Severity', fontsize=18)
plt.xlabel('Day', fontsize=18)
plt.ylabel('Number of Accidents', fontsize=18)
plt.xticks(ticks=range(7), fontsize=16, rotation=0)
plt.legend(fontsize=14,loc='upper left', ncol=2)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Group by the "Accident Type Desc" column and count the occurrences
accident_type_counts = merged_data["accident"]['Accident Type Desc'].value_counts()

# Filter the accident types to only include those with a proportion greater than 1%
filtered_accident_type_counts = accident_type_counts[accident_type_counts / accident_type_counts.sum() > 0.01]


# Plotting the pie chart
plt.figure(figsize=(15, 8))
filtered_accident_type_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('viridis', len(accident_type_counts)))
plt.title('Distribution of Accident Types (Greater than 1%)', fontsize=18)
plt.ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
# Merging the Accident dataframe with the Node dataframe on the "ACCIDENT_NO" column to get the "POSTCODE_NO" column
merged_df = merged_data["accident"].merge(merged_data["Node"][['ACCIDENT_NO', 'POSTCODE_NO', 'Lat','Long','LGA_NAME']], on='ACCIDENT_NO', how='left')

# Renaming the POSTCODE_NO column to POSTCODE for clarity
merged_df.rename(columns={'POSTCODE_NO': 'POSTCODE'}, inplace=True)

# Display the first few rows of the merged dataframe to confirm
merged_df.head()

In [ ]:
# Loading the suburb dataset from the provided Excel file
suburb_df = pd.read_excel('suburb.xlsx')

# Merging the merged_df with the suburb_df on the "POSTCODE" column to get the corresponding suburb (locality)
merged_df_with_suburb = merged_df.merge(suburb_df, left_on='POSTCODE', right_on='postcode', how='left')

# Picking only the first suburb for each accident record
merged_df_with_suburb = merged_df_with_suburb.drop_duplicates(subset='ACCIDENT_NO', keep='first')

# Renaming the locality column to SUBURB for clarity
merged_df_with_suburb.rename(columns={'locality': 'SUBURB'}, inplace=True)

# Display the first few rows of the merged dataframe to confirm
merged_df_with_suburb[['ACCIDENT_NO', 'POSTCODE', 'SUBURB']].head()

In [ ]:
# Grouping by LGA_NAME and counting the number of accidents
lga_accident_counts = merged_df.groupby('LGA_NAME').size().sort_values(ascending=False)

# Displaying the top 10 LGA_NAMEs with the most accidents
top_10_lga = lga_accident_counts.head(10)
top_10_lga

In [ ]:
#copy dataframe
merged_df_2 = merged_data["accident"]

# Mapping the severity values to their respective descriptions
severity_mapping = {
    1: 'Fatal accident',
    2: 'Serious injury accident',
    3: 'Other injury accident',
    4: 'Non injury accident'
}
merged_df_2['SEVERITY'] =  merged_df_2['SEVERITY'].map(severity_mapping)

# Creating a crosstab between 'Light Condition Desc' and 'SEVERITY'
cross_tab = pd.crosstab(merged_df_2['Light Condition Desc'], merged_df_2['SEVERITY'])

# Convert counts to percentages by dividing each column by the sum of that column
cross_tab_percentage = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab_percentage, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
plt.title('Relationship between Light Condition and Severity', fontsize=16)
plt.xlabel('Severity', fontsize=14)
plt.ylabel('Light Condition', fontsize=14)
plt.show()

In [ ]:
# Creating a crosstab between 'Light Condition Desc' and 'SEVERITY'
cross_tab_speedzone = pd.crosstab(filtered_speed_zone['SPEED_ZONE'], filtered_speed_zone['SEVERITY'])

# Convert counts to percentages by dividing each column by the sum of that column
cross_tab_percentage_speedzone = cross_tab_speedzone.div(cross_tab_speedzone.sum(axis=1), axis=0) * 100

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab_percentage_speedzone, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
plt.title('Relationship between Speed Zone and Severity', fontsize=16)
plt.xlabel('Severity', fontsize=14)
plt.ylabel('Speed zone', fontsize=14)
plt.show()

In [ ]:
# Merging the Accident dataframe with the Atmospheric_cond dataframe on the "ACCIDENT_NO" column to get the 'Atmosph Cond Desc' column
merged_df_Atmospheric = pd.merge(merged_data["accident"], merged_data["atmospheric_cond"][['ACCIDENT_NO', 'Atmosph Cond Desc']], on='ACCIDENT_NO', how='inner')

# Display the first few rows of the merged dataframe to confirm
merged_df_Atmospheric.head()

In [ ]:
# Creating a crosstab between 'Atmosph Cond Desc' and 'SEVERITY'
cross_tab = pd.crosstab(merged_df_Atmospheric['Atmosph Cond Desc'], merged_df_Atmospheric['SEVERITY'])

# Convert counts to percentages by dividing each column by the sum of that column
cross_tab_percentage = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab_percentage, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
plt.title('Relationship between Atmospheric Condition and Severity', fontsize=16)
plt.xlabel('Severity', fontsize=14)
plt.ylabel('Light Condition', fontsize=14)
plt.show()

In [ ]:
# Merging the Accident dataframe with the Atmospheric_cond dataframe on the "ACCIDENT_NO" column to get the 'Surface Cond Desc' column
merged_df_Surface = pd.merge(merged_data["accident"], merged_data["road_surface_cond"][['ACCIDENT_NO', 'Surface Cond Desc']], on='ACCIDENT_NO', how='inner')

# Display the first few rows of the merged dataframe to confirm
merged_df_Surface.head()

In [ ]:
# Creating a crosstab between 'Atmosph Cond Desc' and 'SEVERITY'
cross_tab = pd.crosstab(merged_df_Surface['Surface Cond Desc'], merged_df_Surface['SEVERITY'])

# Convert counts to percentages by dividing each column by the sum of that column
cross_tab_percentage = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab_percentage, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
plt.title('Relationship between Surface Condition and Severity', fontsize=16)
plt.xlabel('Severity', fontsize=14)
plt.ylabel('Light Condition', fontsize=14)
plt.show()

In [21]:
import re
# Merge with Road_Surface_Cond table
merged_df_1 = pd.merge(merged_data["accident"], merged_data["road_surface_cond"][['ACCIDENT_NO', 'SURFACE_COND']], on='ACCIDENT_NO', how='left')

# Merge with Atmospheric_Cond table
merged_df_2 = pd.merge(merged_df_1, merged_data["atmospheric_cond"][['ACCIDENT_NO', 'ATMOSPH_COND']], on='ACCIDENT_NO', how='left')

# Merge with Node table
merged_df_3 = pd.merge(merged_df_2, merged_data["Node"][['ACCIDENT_NO', 'LGA_NAME']], on='ACCIDENT_NO', how='left')

# Extract the hour from the `ACCIDENTTIME` column using the correct format and create a new column
# Replace "." with ":" in the ACCIDENTTIME column
merged_df_3['ACCIDENTTIME'] = merged_df_3['ACCIDENTTIME'].str.replace('.', ':')

# Extract the hour
merged_df_3['HOUR'] = merged_df_3['ACCIDENTTIME'].str.split(':').str[0]

# Convert the HOUR column to numeric
merged_df_3['HOUR'] = pd.to_numeric(merged_df_3['HOUR'], errors='coerce')


# Filter the dataframe to only include the desired columns
final_df = merged_df_3[['HOUR', 'DAY_OF_WEEK', 'LIGHT_CONDITION', 'SPEED_ZONE', 'SURFACE_COND', 'ATMOSPH_COND', 'LGA_NAME', 'SEVERITY']]

# Drop rows with missing values
cleaned_df = final_df.dropna()

# Display the first few rows of the final dataframe
print(cleaned_df.head())

C:\Users\buida\AppData\Local\Temp/ipykernel_21356/3339354653.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_df_3['ACCIDENTTIME'] = merged_df_3['ACCIDENTTIME'].str.replace('.', ':')


        HOUR  DAY_OF_WEEK  LIGHT_CONDITION  SPEED_ZONE  SURFACE_COND  \
102694   0.0            1                3          60           1.0   
102695  19.0            6                1          60           1.0   
102696  15.0            5                1          60           1.0   
102697  12.0            2                1          60           1.0   
102698  19.0            5                1          60           1.0   

        ATMOSPH_COND   LGA_NAME  SEVERITY  
102694           1.0      YARRA         2  
102695           1.0      YARRA         3  
102696           1.0  MELBOURNE         3  
102697           1.0  MELBOURNE         3  
102698           1.0      YARRA         3  


In [17]:
output_file_path = "Clean_data.xlsx"
cleaned_df.to_excel(output_file_path)

HOUR               0
DAY_OF_WEEK        0
LIGHT_CONDITION    0
SPEED_ZONE         0
SURFACE_COND       0
ATMOSPH_COND       0
LGA_NAME           0
SEVERITY           0
dtype: int64
